# Calculate and save WRF predictor variables over entire spatiotemporal domain
Last updated: Kevin Varga, 05/20/2024

**Inputs:**
* Pre-processed WRF files of daily temperature, precipitation, relative humidity, and wind
* Pre-processed WRF files of shortwave and net radiation, soil moisture, pressure, and actual evapotranspiration <br>

**Outputs:**
* /home/sbarc/students/varga/nasa/ch1/data/predictors/temp90mean.nc - mean 90 day temperature <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/precip30sum.nc - sum 30 day precipitation <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/precip90sum.nc - sum 90 day precipitation <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/rh150mean.nc - mean 150 day relative humidity <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/rad150mean.nc - mean 150 day incoming radiation <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/somo7mean.nc - mean 7 day soil moisture <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/daylength.nc - length of the day <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/eto90sum.nc - sum 90 day reference evapotranspiration <br>
* /home/sbarc/students/varga/nasa/ch1/data/predictors/cwd90sum.nc - sum 90 day climatic water deficit <br>

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [2]:
# path in for wrf files that were previously processed by Charles Jones, UCSB
# includes daily mean temp, rh, precip, and wind, saved in yearly outputs
wrf_ch_path = '/home/sbarc/wrf/wrf401/sbareg/sbcwrf/'

# path in for wrf files that were previously processed by kevin varga
# includes aet, ea, es, netrad, psfc, smois, and swdnb
wrf_vars_path = '/home/sbarc/students/varga/nasa/ch1/data/wrf/vars/'

# path to save predictor outputs
wrf_out = '/home/sbarc/students/varga/nasa/ch1/data/predictors/'
wrf_out_daily = '/home/sbarc/students/varga/nasa/ch1/data/predictors/daily/'

In [3]:
file_in = 'land_mask.nc'
# open data array containing 1 for land and 0 for water
land_mask = xr.open_dataarray(wrf_vars_path + file_in)
# extract lat/lon values
wrf_lats = land_mask['XLAT'].values[:,0]
wrf_lons = land_mask['XLONG'].values[0,:]
# assign lat/lon coordinates as spatial dimensions and clean up
land_mask['south_north'] = wrf_lats
land_mask['west_east'] = wrf_lons
land_mask = land_mask.rename({'south_north':'latitude', 'west_east':'longitude'})
ex_coords = ['XLAT','XLONG','XTIME']
land_mask = land_mask.drop(ex_coords)

# create mask for land
land_mask = (land_mask >=1)

## temp90mean

In [5]:
%%time
dir_in = 'dly_airt/'
file_in = '*mean*.nc'
file_out = 'temp90mean'

# create list of all mean temperature wrf files
wrf_list = sorted(list(Path(wrf_ch_path + dir_in).glob(file_in)))

# open all files into a dataset
da = xr.open_mfdataset(wrf_list)

# assign lat/lon coordinates as spatial dimensions and clean up
da['south_north'] = wrf_lats
da['west_east'] = wrf_lons
da = da.rename({'south_north':'latitude', 'west_east':'longitude', 'times':'time'})
ex_coords = ['XLAT','XLONG']
da = da.drop(ex_coords)

# apply land-sea mask
da = da['airtmean'].where(land_mask, drop=True)

# load variable out of dask array to remove 2017-08-28 bad values
da = da.load()
da.loc['2017-08-28'] = np.nan

# create temp array for later use in eto calculation
temp_array = da.data

# calculate 90-day moving averages
da = da.rolling(time=90, min_periods=86).mean()

# save daily output for use in model validation
da.name = file_out
da.to_netcdf(wrf_out_daily + file_out + '_daily.nc')

# resample to values on the 1st and the 15th for dataset creation
da = da.resample(time='SMS').nearest()

# save semi-monthly output for dataset creation
da.to_netcdf(wrf_out + file_out + '.nc')

CPU times: user 1min 24s, sys: 30.2 s, total: 1min 54s
Wall time: 2min


## precip30sum and precip90sum

In [8]:
%%time
dir_in = 'dly_prec/'
file_in = '*.nc'
precip30_file_out = 'precip30sum'
precip90_file_out = 'precip90sum'

# create list of all mean temperature wrf files
wrf_list = sorted(list(Path(wrf_ch_path + dir_in).glob(file_in)))

# open all files into a dataset
da = xr.open_mfdataset(wrf_list)

# assign lat/lon coordinates as spatial dimensions and clean up
da['south_north'] = wrf_lats
da['west_east'] = wrf_lons
da = da.rename({'south_north':'latitude', 'west_east':'longitude', 'times':'time'})
ex_coords = ['XLAT','XLONG']
da = da.drop(ex_coords)

# apply land-sea mask
da = da['precsum'].where(land_mask, drop=True)

# load variable out of dask array to remove 2017-08-28 bad data, found through plotting
da = da.load()
da.loc['2017-08-28'] = np.nan

# calculate daily rainfall from accumulated rainfall
array = np.empty([len(da['time']),len(da['latitude']),len(da['longitude'])])
for i in range(len(da['time'])):
    if da['time.month'][i].values == 7 and da['time.day'][i].values == 1:
        array[i,:,:] = da[i,:,:]
    else:
        array[i,:,:] = da[i,:,:] - da[i-1,:,:]

da.data = array

# calculate 30-day accumulations
da30 = da.rolling(time=30, min_periods=28).sum()

# calculate 90-day accumulations
da90 = da.rolling(time=90, min_periods=86).sum()

# save daily output
da30.name = precip30_file_out
da90.name = precip90_file_out
da30.to_netcdf(wrf_out_daily + precip30_file_out + '_daily.nc')
da90.to_netcdf(wrf_out_daily + precip90_file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da30 = da30.resample(time='SMS').nearest()
da90 = da90.resample(time='SMS').nearest()

# save semi-monthly output
da30.to_netcdf(wrf_out + precip30_file_out + '.nc')
da90.to_netcdf(wrf_out + precip90_file_out + '.nc')

CPU times: user 4min 10s, sys: 1min 1s, total: 5min 11s
Wall time: 4min 28s


## rh150mean

In [6]:
%%time
dir_in = 'dly_relhum/'
file_in = '*mean*.nc'
file_out = 'rh150mean'

# create list of all mean temperature wrf files
wrf_list = sorted(list(Path(wrf_ch_path + dir_in).glob(file_in)))

# open all files into a dataset
da = xr.open_mfdataset(wrf_list)

# assign lat/lon coordinates as spatial dimensions and clean up
da['south_north'] = wrf_lats
da['west_east'] = wrf_lons
da = da.rename({'south_north':'latitude', 'west_east':'longitude', 'times':'time'})
ex_coords = ['XLAT','XLONG']
da = da.drop(ex_coords)

# apply land-sea mask
da = da['rhmean'].where(land_mask, drop=True)

# load variable out of dask array to remove 2017-08-28 bad values
da = da.load()
da.loc['2017-08-28'] = np.nan

# calculate 150-day moving averages
da = da.rolling(time=150, min_periods=142).mean()

# save daily output
da.name = file_out
da.to_netcdf(wrf_out_daily + file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly output
da.to_netcdf(wrf_out + file_out + '.nc')

CPU times: user 1min 16s, sys: 43 s, total: 1min 59s
Wall time: 2min 34s


## rad150mean

In [7]:
%%time
file_in = 'swdnb.nc'
file_out = 'rad150mean'

# read in shortwave radiation file covering all years
da = xr.open_dataarray(wrf_vars_path + file_in)

# resample to fill missing days with nan
da = da.resample(time='D').ffill(0)

# apply land-sea mask
da = da.where(land_mask, drop=True)

# calculate 150-day moving averages
da = da.rolling(time=150, min_periods=142).mean()

# save daily output
da.name = file_out
da.to_netcdf(wrf_out_daily + file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly output
da.to_netcdf(wrf_out + file_out + '.nc')

## somo7mean

In [8]:
%%time
file_in = 'smois.nc'
file_out = 'somo7mean'

# read in shortwave radiation file covering all years
da = xr.open_dataarray(wrf_vars_path + file_in)

# resample and interpolate to fill sporadic missing days
da = da.resample(time='D').interpolate('quadratic')

# apply land-sea mask
da = da.where(land_mask, drop=True)

# calculate 7-day moving averages
da = da.rolling(time=7).mean()

# save daily output
da.name = file_out
da.to_netcdf(wrf_out_daily + file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly output
da.name = file_out
da.to_netcdf(wrf_out + file_out + '.nc')

CPU times: user 6min 35s, sys: 1min 59s, total: 8min 35s
Wall time: 5min 57s


## daylength

In [9]:
# day length function from:
# https://gist.github.com/anttilipp/ed3ab35258c7636d87de6499475301ce

def daylength(dayOfYear, lat):
    """Computes the length of the day (the time between sunrise and
    sunset) given the day of the year and latitude of the location.
    Function uses the Brock model for the computations.
    For more information see, for example,
    Forsythe et al., "A model comparison for daylength as a
    function of latitude and day of year", Ecological Modelling,
    1995.
    Parameters
    ----------
    dayOfYear : int
        The day of the year. 1 corresponds to 1st of January
        and 365 to 31st December (on a non-leap year).
    lat : float
        Latitude of the location in degrees. Positive values
        for north and negative for south.
    Returns
    -------
    d : float
        Daylength in hours.
    """
    latInRad = np.deg2rad(lat)
    declinationOfEarth = 23.45*np.sin(np.deg2rad(360.0*(283.0+dayOfYear)/365.0))
    if -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) <= -1.0:
        return 24.0
    elif -np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth)) >= 1.0:
        return 0.0
    else:
        hourAngle = np.rad2deg(np.arccos(-np.tan(latInRad) * np.tan(np.deg2rad(declinationOfEarth))))
        return 2.0*hourAngle/15.0

In [10]:
%%time
file_out = 'daylength'

# determine day of the year for all sms frequency dates in previous data array
doy = da['time'].dt.dayofyear
# separate out unique values of doy and latitude
doy_u = pd.unique(doy)
lats = pd.unique(da['latitude'])

# create array to store daylength calculation
dl = np.empty((len(doy_u),len(lats)))
# loop through unique doy and latitude to calculate daylength
for i, dayofyear in enumerate(doy_u):
    for j, latitude in enumerate(lats):
        dl[i,j] = daylength(dayofyear, latitude)

# create array to fill in daylength values for all dates
dl_grid = np.empty((len(da['time']),len(da['latitude'])))

# loop through daylength calculations and fill in grid array where doy matches
for i,value in enumerate(doy_u):
    idx = np.where(doy == value)
    for j in idx:
        dl_grid[j,:] = dl[i,:]
        
# create longitude dimension and repeat latitudinal values across dimension
dl_grid = np.repeat(dl_grid[:,:, np.newaxis], len(da['longitude']), axis=2)

# create new data array with daylength values
da = xr.DataArray(dl_grid, coords = (da['time'], da['latitude'], da['longitude']))

# apply land-sea mask
da = da.where(land_mask, drop=True)

# save daily output
da.name = file_out
da.to_netcdf(wrf_out_daily + file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly output
da.name = file_out
da.to_netcdf(wrf_out + file_out + '.nc')

CPU times: user 26.8 s, sys: 9.58 s, total: 36.4 s
Wall time: 36.2 s


## eto90sum and cwd90sum
Uses temp_array from temp90mean calculation

In [4]:
def wind_speed_2m(ws, z):
    """
    Convert wind speed measured at different heights above the soil
    surface to wind speed at 2 m above the surface, assuming a short grass
    surface.
    Based on FAO equation 47 in Allen et al (1998).
    :param ws: Measured wind speed [m s-1]
    :param z: Height of wind measurement above ground surface [m]
    :return: Wind speed at 2 m above the surface [m s-1]
    :rtype: float
    """
    return ws * (4.87 / math.log((67.8 * z) - 5.42))

In [5]:
def delta_svp(t):
    """
    Estimate the slope of the saturation vapour pressure curve at a given
    temperature.
    Based on equation 13 in Allen et al (1998). If using in the Penman-Monteith
    *t* should be the mean air temperature.
    :param t: Air temperature [deg C]. Use mean air temperature for use in
        Penman-Monteith.
    :return: Saturation vapour pressure [kPa degC-1]
    :rtype: float
    """
    tmp = 4098 * (0.6108 * np.exp((17.27 * t) / (t + 237.3)))
    return tmp / np.power((t + 237.3), 2)

In [6]:
def psy_const(atmos_pres):
    """
    Calculate the psychrometric constant.
    This method assumes that the air is saturated with water vapour at the
    minimum daily temperature. This assumption may not hold in arid areas.
    Based on equation 8, page 95 in Allen et al (1998).
    :param atmos_pres: Atmospheric pressure [kPa]. Can be estimated using
        ``atm_pressure()``.
    :return: Psychrometric constant [kPa degC-1].
    :rtype: float
    """
    return 0.000665 * atmos_pres

In [7]:
def fao56_penman_monteith(net_rad, t, ws, svp, avp, delta_svp, psy, shf=0.0):
    """
    Estimate reference evapotranspiration (ETo) from a hypothetical
    short grass reference surface using the FAO-56 Penman-Monteith equation.
    Based on equation 6 in Allen et al (1998).
    :param net_rad: Net radiation at crop surface [MJ m-2 day-1]. If
        necessary this can be estimated using ``net_rad()``.
    :param t: Air temperature at 2 m height [deg Kelvin].
    :param ws: Wind speed at 2 m height [m s-1]. If not measured at 2m,
        convert using ``wind_speed_at_2m()``.
    :param svp: Saturation vapour pressure [kPa]. Can be estimated using
        ``svp_from_t()''.
    :param avp: Actual vapour pressure [kPa]. Can be estimated using a range
        of functions with names beginning with 'avp_from'.
    :param delta_svp: Slope of saturation vapour pressure curve [kPa degC-1].
        Can be estimated using ``delta_svp()``.
    :param psy: Psychrometric constant [kPa deg C]. Can be estimatred using
        ``psy_const_of_psychrometer()`` or ``psy_const()``.
    :param shf: Soil heat flux (G) [MJ m-2 day-1] (default is 0.0, which is
        reasonable for a daily or 10-day time steps). For monthly time steps
        *shf* can be estimated using ``monthly_soil_heat_flux()`` or
        ``monthly_soil_heat_flux2()``.
    :return: Reference evapotranspiration (ETo) from a hypothetical
        grass reference surface [mm day-1].
    :rtype: float
    """
    a1 = (0.408 * (net_rad - shf) * delta_svp /
          (delta_svp + (psy * (1 + 0.34 * ws))))
    a2 = (900 * ws / t * (svp - avp) * psy /
          (delta_svp + (psy * (1 + 0.34 * ws))))
    return a1 + a2

In [8]:
%%time
# set file in names
aet_file_in = 'aet.nc'
pres_file_in = 'psfc.nc'
netrad_file_in = 'netrad.nc'
ea_file_in = 'ea.nc'
es_file_in = 'es.nc'

# wind file inputs from charles processing
dir_in = 'dly_winds/'
wnd_file_in = '*mean*.nc'

# output names
ds_out = 'cwd_ds.nc'
eto_file_out = 'eto90sum'
cwd_file_out = 'cwd90sum'

# create list of all mean daily wind wrf files
wrf_list = sorted(list(Path(wrf_ch_path + dir_in).glob(wnd_file_in)))

# open all files into a dataset
ds = xr.open_mfdataset(wrf_list)

# assign lat/lon coordinates as spatial dimensions and clean up
ds['south_north'] = wrf_lats
ds['west_east'] = wrf_lons
ds = ds.rename({'south_north':'latitude', 'west_east':'longitude', 'times':'time'})
ex_coords = ['XLAT','XLONG']
ds = ds.drop(ex_coords)

# apply land-sea mask
ds = ds.where(land_mask, drop=True)

# calculate wind speed using u and v components and drop components
ds['wspd'] = (['time', 'latitude', 'longitude'], np.sqrt(ds['u10mean'].to_numpy()**2 + ds['v10mean'].to_numpy()**2))
ds = ds.drop(['u10mean','v10mean'])

# convert winds from 10m height to 2m height and save in dataset
wspd2m = wind_speed_2m(ds['wspd'].to_numpy(), 10)
ds['wspd'] = (['time', 'latitude', 'longitude'], wspd2m)
print('calculated winds')

# save temperature to dataset, 
# which was previously saved as temp_array from temp90mean section
ds['temp'] = (['time', 'latitude', 'longitude'], temp_array)

# convert temp_array to celcius to be used in delta_svp calculation
temp_array = temp_array - 273.15

# calculate slope of saturation vapor pressure curve at given temperature
svp = delta_svp(temp_array)
ds['svp'] = (['time', 'latitude', 'longitude'], svp)
print('calculated svp')

# read in daily pressure netcdf
da = xr.open_dataarray(wrf_vars_path + pres_file_in)
# resample and interpolate sporadic missing days
da = da.resample(time='D').interpolate('quadratic')
# apply land-sea mask
da = da.where(land_mask, drop=True)

# calculate psychrometric constant at given pressure
psy = psy_const(da.data)
ds['psy'] = (['time', 'latitude', 'longitude'], psy)
print('calculated psy')

# read in daily netrad netcdf
da = xr.open_dataarray(wrf_vars_path + netrad_file_in)
# resample to fill missing days with nan
da = da.resample(time='D').interpolate('quadratic')
# apply land-sea mask
da = da.where(land_mask, drop=True)
# feed data into dataset
ds['netrad'] = (['time', 'latitude', 'longitude'], (da.data * 0.0864))
print('done with netrad')

# read in daily actual vapor pressure (ea) netcdf
da = xr.open_dataarray(wrf_vars_path + ea_file_in)
# resample to fill missing days with nan
da = da.resample(time='D').interpolate('quadratic')
# apply land-sea mask
da = da.where(land_mask, drop=True)
# feed data into dataset
ds['ea'] = (['time', 'latitude', 'longitude'], (da.data / 1000))
print('done with ea')

# read in daily saturation vapor pressure (es) netcdf
da = xr.open_dataarray(wrf_vars_path + es_file_in)
# resample to fill missing days with nan
da = da.resample(time='D').interpolate('quadratic')
# apply land-sea mask
da = da.where(land_mask, drop=True)
# feed data into dataset
ds['es'] = (['time', 'latitude', 'longitude'], (da.data / 1000))
print('done with es')

# calculate daily reference evapotranspiration using penman monteith equation and save in dataset
eto1sum = fao56_penman_monteith(ds['netrad'].to_numpy(), ds['temp'].to_numpy(), ds['wspd'].to_numpy(), \
                                ds['es'].to_numpy(), ds['ea'].to_numpy(), ds['svp'].to_numpy(), ds['psy'].to_numpy(), shf=0.0)
ds['eto'] = (['time', 'latitude', 'longitude'], eto1sum)
print('done with eto')

# read in daily actual evapotranspiration netcdf
da = xr.open_dataarray(wrf_vars_path + aet_file_in)
# resample and interpolate sporadic missing days
da = da.resample(time='D').interpolate('quadratic')
# apply land-sea mask
da = da.where(land_mask, drop=True)
# feed data into dataset
ds['aet'] = (['time', 'latitude', 'longitude'], da.data)

# calculate daily climatic water deficit
ds['cwd'] = (['time', 'latitude', 'longitude'], (ds['eto'].to_numpy() - ds['aet'].to_numpy()))

# save dataset as netcdf
# ds.to_netcdf(wrf_out + ds_out)
# print('saved dataset')

# calculate 90-day sum for reference evapotranspiration
da = ds['eto'].rolling(time=90, min_periods=85).sum()

# save daily outputs
da.name = eto_file_out
#da.to_netcdf(wrf_out_daily + eto_file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly outputs
#da.to_netcdf(wrf_out + eto_file_out + '.nc')
print('saved eto')

# calculate 90-day sum for climatic water deficit
da = ds['cwd'].rolling(time=90, min_periods=85).sum()

# save daily outputs
da.name = cwd_file_out
da.to_netcdf(wrf_out_daily + cwd_file_out + '_daily.nc')

# resample to values on the 1st and the 15th
da = da.resample(time='SMS').nearest()

# save semi-monthly outputs
da.to_netcdf(wrf_out + cwd_file_out + '.nc')
print('saved cwd')

calculated winds


NameError: name 'temp_array' is not defined